In [ ]:
from fastai import *
from fastai.vision import *
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import dlib
import subprocess
import random
import os
bs=8
torch.cuda.is_available()
torch.cuda.set_device(0)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
## Declaring path of dataset
path_img = Path('/root/model_training/intern_images')

In [ ]:
#function to create list of cv2-loaded images in a given directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),1)
        if img is not None:
            images.append(img)
    return images

#function to convert cv2-loaded image into RGB
def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
# model to use to locate faces
haar_cascade_face = cv2.CascadeClassifier('./data/haarcascade/haarcascade_frontalface_default.xml')

for dirname, dirnames, filenames in os.walk('./intern_images'):
    for subdirname in dirnames:
        #print(os.path.join(dirname, subdirname))
        imgs = load_images_from_folder(os.path.join(dirname, subdirname))
        print('loaded images for '+ subdirname)
        for img in imgs:
            print(img.shape)
            width = 720
            height = 1080
            # for if you want to preserve aspect ratio
            # scale_percent = 25 # percent of original size
            # width = int(img.shape[1] * scale_percent / 100)
            # height = int(img.shape[0] * scale_percent / 100)
            dim = (width, height)
            # resize image
            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            faces = haar_cascade_face.detectMultiScale(resized, scaleFactor = 1.2, minNeighbors = 5)
            for (x,y,w,h) in faces:
                 cv2.rectangle(resized, (x, y), (x+w, y+h), (0, 255, 0), 2)
            plt.imshow(convertToRGB(resized))

In [ ]:
img_raw = cv2.imread(path_img+'max/max_001.JPG',1)
type(img_raw)
img_raw.shape
plt.imshow(img_raw)
cv2.imwrite(os.path.join(path , 'waka.jpg'), img)
cv2.waitKey(0)

In [ ]:
## applying transforms
tfms_list = [rotate(degrees=270, p=1),jitter(magnitude=(random.randrange(-3,3)/100), p=0.25), 
            contrast(scale=(0.5, 2.), p=0.5), brightness(change=(0.1, 0.9), p=0.5)]
tfms_list = [rotate(degrees=270, p=1)]
tfms = [tfms_list, tfms_list]

## Loading data 
data = ImageDataBunch.from_folder(path=path_img, train='/', valid_pct=0.07, ds_tfms=tfms, bs=bs, size=(540,720))

## Normalizing data based on ImageNet parameters
data.normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=2)

In [ ]:
print(data.classes)
len(data.classes),data.c

In [ ]:
face_detector = dlib.get_frontal_face_detector()
for file_name in data.items:
    img = dlib.load_rgb_image(str(file_name))
    detected_faces = face_detector(img)
    print("Found {} faces in the image file {}".format(len(detected_faces), file_name))

In [ ]:
def get_ex(): return open_image(path_img/'max/max_001.JPG')

def plots_f(rows, cols, width, height, **kwargs):
    [get_ex().apply_tfms(tfms[0], **kwargs).show(ax=ax) for i,ax in enumerate(plt.subplots(
        rows,cols,figsize=(width,height))[1].flatten())]


In [ ]:
## To create a ResNET 50 with pretrained weights
learn = cnn_learner(data, models.resnet50, metrics=accuracy)
print(learn)

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
print('Training results:  ' + str(learn.validate(learn.data.train_dl)))

In [ ]:
print('Validation results: ' + str(learn.validate(learn.data.valid_dl)))

In [ ]:
preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(4, figsize=(5,5))

In [ ]:
interp.plot_confusion_matrix(figsize=(5,5), dpi=60)

In [ ]:
learn.lr_find()
learn.recorder.plot()